Trabajo de curso

In [8]:
import cv2
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np

MARGIN = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54) # vibrant green

def draw_landmarks_on_image(rgb_image, detection_result):
    hand_landmarks_list = detection_result.hand_landmarks
    handedness_list = detection_result.handedness
    annotated_image = np.copy(rgb_image)

    # Loop through the detected hands to visualize.
    for idx in range(len(hand_landmarks_list)):
        hand_landmarks = hand_landmarks_list[idx]
        handedness = handedness_list[idx]

        # Draw the hand landmarks.
        hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
        hand_landmarks_proto.landmark.extend([
        landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
        ])
        solutions.drawing_utils.draw_landmarks(
        annotated_image,
        hand_landmarks_proto,
        solutions.hands.HAND_CONNECTIONS,
        solutions.drawing_styles.get_default_hand_landmarks_style(),
        solutions.drawing_styles.get_default_hand_connections_style())

        # Get the top left corner of the detected hand's bounding box.
        height, width, _ = annotated_image.shape
        x_coordinates = [landmark.x for landmark in hand_landmarks]
        y_coordinates = [landmark.y for landmark in hand_landmarks]
        text_x = int(min(x_coordinates) * width)
        text_y = int(min(y_coordinates) * height) - MARGIN

        # Draw handedness (left or right hand) on the image.
        cv2.putText(annotated_image, f"{handedness[0].category_name}",
                    (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX,
                    FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv2.LINE_AA)

    return annotated_image

import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision


# Cámara
vid = cv2.VideoCapture(0)

# Crea objeto FaceLandmarker 
# Obtener archivo .task y modificar la ruta 
base_options = python.BaseOptions(model_asset_path='D:/david/descargas/hand_landmarker.task')
options = vision.HandLandmarkerOptions(base_options=base_options,
                                       num_hands=2)
detector = vision.HandLandmarker.create_from_options(options)
  
while(True):      
    # fotograma a fotograma
    ret, frame = vid.read()
  
    if ret:          
        # Adapta formato
        image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)

        # Detección
        detection_result = detector.detect(image)

        # Visualiza resultado
        annotated_image = draw_landmarks_on_image(image.numpy_view(), detection_result)

        # Muestra fotograma
        cv2.imshow('Vid', annotated_image)
    
    # Detenemos pulsado ESC
    if cv2.waitKey(20) == 27:
        break
  
# Libera el objeto de captura
vid.release()
# Destruye ventanas
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [11]:
from cvzone.HandTrackingModule import HandDetector
import cv2

# Cámara
vid = cv2.VideoCapture(0)

# Initializa detector
detector = HandDetector(staticMode=False, maxHands=2, modelComplexity=1, detectionCon=0.5, minTrackCon=0.5)

while(True):      
    # Fotograma a fotograma
    ret, frame = vid.read()

    # Búsqueda de manos
    # 'draw' a True indica si se dibujan sobre la imagen 
    # 'flipType' a True para tratar la imagen reflejada
    hands, frame = detector.findHands(frame, draw=True, flipType=True)
    
    # Si hay manos detectadas
    if hands:
        # datos primera mano
        hand1 = hands[0]  
        lmList1 = hand1["lmList"]  # 21 landmarks
        bbox1 = hand1["bbox"]  # Contenedor (x,y,w,h)
        center1 = hand1['center']  # Centro
        handType1 = hand1["type"]  # identifica si es la mano derecha o izquierda

        # Contabiliza dedos extendidos de la mano
        fingers1 = detector.fingersUp(hand1)
        print(f'H1 = {fingers1.count(1)}', end=" ")  


        # Calcula distancia entre dos elementos concretos de la mano, dibujando segmento entre ellos
        length, info, frame = detector.findDistance(lmList1[8][0:2], lmList1[12][0:2], frame, color=(255, 0, 0),scale=10)

        # Segunda mano presente?
        if len(hands) == 2:
            # Datos segunda mano
            hand2 = hands[1]
            lmList2 = hand2["lmList"]
            bbox2 = hand2["bbox"]
            center2 = hand2['center']
            handType2 = hand2["type"]

            # Contabiliza dedos extendidos de la mano
            fingers2 = detector.fingersUp(hand2)
            print(f'H2 = {fingers2.count(1)}', end=" ")

            # Calcula distancia entre dos elementos concretos de la mano, dibujando segmento entre ellos
            #length, info, frame = detector.findDistance(lmList2[8][0:2], lmList2[12][0:2], frame, color=(255, 0, 0),scale=10)

        print("\n")  

    # Display the image in a window
    cv2.imshow("Image", frame)

    # Detenemos pulsado ESC
    if cv2.waitKey(20) == 27:
        break

# Libera el objeto de captura
vid.release()
# Destruye ventanas
cv2.destroyAllWindows()

H1 = 4 

H1 = 4 

H1 = 4 

H1 = 4 

H1 = 4 

H1 = 4 

H1 = 4 

H1 = 4 

H1 = 4 

H1 = 4 

H1 = 4 

H1 = 4 

H1 = 4 

H1 = 4 

H1 = 4 

H1 = 4 

H1 = 4 

H1 = 4 

H1 = 5 

H1 = 5 

H1 = 5 

H1 = 5 

H1 = 5 

H1 = 5 

H1 = 5 H2 = 5 

H1 = 4 H2 = 5 

H1 = 5 H2 = 5 

H1 = 5 

H1 = 5 

H1 = 5 H2 = 5 

H1 = 5 H2 = 5 

H1 = 5 

H1 = 5 

H1 = 5 H2 = 5 

H1 = 5 

H1 = 5 H2 = 5 

H1 = 5 

H1 = 5 

H1 = 5 H2 = 5 

H1 = 5 

H1 = 4 

H1 = 5 

H1 = 4 

H1 = 5 

H1 = 5 

H1 = 5 

H1 = 5 

H1 = 5 

H1 = 5 

H1 = 5 

H1 = 5 

H1 = 5 

H1 = 5 

H1 = 5 

H1 = 5 

H1 = 5 

H1 = 5 

H1 = 5 

H1 = 5 

H1 = 5 

H1 = 5 

H1 = 5 

H1 = 4 H2 = 5 

H1 = 4 H2 = 5 

H1 = 5 

H1 = 5 

H1 = 5 

H1 = 5 

H1 = 5 

H1 = 5 

H1 = 5 

H1 = 5 

H1 = 5 

H1 = 5 

H1 = 5 

H1 = 5 

H1 = 5 

H1 = 5 

H1 = 5 

H1 = 5 

H1 = 5 

H1 = 5 H2 = 5 

H1 = 5 

H1 = 5 H2 = 5 

H1 = 5 

H1 = 5 

H1 = 5 

H1 = 5 

H1 = 5 

H1 = 5 

H1 = 5 

H1 = 5 

H1 = 5 

H1 = 5 

H1 = 4 

H1 = 5 

H1 = 4 

H1 = 4 

H1 = 4 

H1 = 4 

H1 = 4 

H1 = 4 

In [1]:
import cv2
from cvzone.HandTrackingModule import HandDetector

# Configuración de la cámara
cap = cv2.VideoCapture(0)
cap.set(3, 1280)  # Ancho de la ventana
cap.set(4, 720)   # Alto de la ventana

# Detector de manos
detector = HandDetector(detectionCon=0.8, maxHands=1)

# Variables para dibujar
drawing = False  # Si está dibujando o no
points = []  # Lista de puntos para almacenar las posiciones del dibujo

while True:
    # Leer el frame de la cámara
    success, img = cap.read()
    if not success:
        break

    # Detectar manos
    hands, img = detector.findHands(img)

    if hands:
        hand = hands[0]  # Solo la primera mano detectada
        lmList = hand["lmList"]  # Lista de posiciones (landmarks)
        
        # Coordenadas del dedo índice (punta)
        x, y = lmList[8][:2]  # Índice

        # Coordenadas del pulgar (punta)
        x_thumb, y_thumb = lmList[4][:2]  # Pulgar

        # Distancia entre el índice y el pulgar
        length, _, _ = detector.findDistance(lmList[8][:2], lmList[4][:2], img)

        # Si la distancia es menor a un umbral, activa el dibujo
        if length < 30:
            drawing = True
            points.append((x, y))  # Agregar punto para el dibujo
        else:
            drawing = False

    # Dibujar los puntos en la pantalla
    for i in range(1, len(points)):
        cv2.line(img, points[i-1], points[i], (0, 0, 255), 5)  # Línea roja entre puntos consecutivos

    # Mostrar el estado de "escritura"
    if drawing:
        cv2.putText(img, "Dibujando...", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)

    # Mostrar la imagen resultante
    cv2.imshow("Dibujo", img)

    # Salir con la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

: 

In [1]:
import cv2
from cvzone.HandTrackingModule import HandDetector

# Configuración de la cámara
cap = cv2.VideoCapture(0)
cap.set(3, 1280)  # Ancho de la ventana
cap.set(4, 720)   # Alto de la ventana

# Detector de manos
detector = HandDetector(detectionCon=0.8, maxHands=1)

# Variables para dibujar
drawing = False  # Si está dibujando o no
points = []  # Lista de puntos para almacenar las posiciones del dibujo

while True:
    # Leer el frame de la cámara
    success, img = cap.read()
    if not success:
        break

    # Detectar manos
    hands, img = detector.findHands(img)

    if hands:
        hand = hands[0]  # Solo la primera mano detectada
        lmList = hand["lmList"]  # Lista de posiciones (landmarks)
        
        # Coordenadas del dedo índice (punta)
        x, y = lmList[8][:2]  # Índice

        # Coordenadas del pulgar (punta)
        x_thumb, y_thumb = lmList[4][:2]  # Pulgar

        # Distancia entre el índice y el pulgar
        length, _, _ = detector.findDistance(lmList[8][:2], lmList[4][:2], img)

        # Si la distancia es menor a un umbral, activa el dibujo
        if length < 50:
            if not drawing:  # Si recién empieza a dibujar
                points.append(None)  # Punto de ruptura
            drawing = True
            points.append((x, y))  # Agregar punto para el dibujo
        else:
            drawing = False

    # Dibujar los puntos en la pantalla
    for i in range(1, len(points)):
        if points[i - 1] is None or points[i] is None:
            continue  # Saltar puntos de ruptura
        cv2.line(img, points[i - 1], points[i], (0, 0, 255), 5)  # Línea roja entre puntos consecutivos

    # Mostrar el estado de "escritura"
    if drawing:
        cv2.putText(img, "Dibujando...", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)

    # Mostrar la imagen resultante
    cv2.imshow("Dibujo", img)

    # Salir con la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [2]:
import cv2
from cvzone.HandTrackingModule import HandDetector
import time  # Para manejar el temporizador

# Configuración de la cámara
cap = cv2.VideoCapture(0)
cap.set(3, 1280)  # Ancho de la ventana
cap.set(4, 720)   # Alto de la ventana

# Detector de manos
detector = HandDetector(detectionCon=0.8, maxHands=1)

# Variables para dibujar
drawing = False  # Si está dibujando o no
points_dict = {}  # Diccionario para almacenar los puntos asociados a colores y grosores

# Lista de colores disponibles
colors = [(0, 0, 255), (0, 255, 0), (255, 0, 0), (255, 255, 0), (255, 0, 255), (0, 255, 255)]  # Rojo, verde, azul, amarillo, magenta, cian
current_color_index = 0  # Índice del color actual
current_color = colors[current_color_index]

# Grosor del dibujo
thicknesses = [2, 10, 25, 60]  # Lista de grosores disponibles
current_thickness_index = 1  # Índice del grosor actual
current_thickness = thicknesses[current_thickness_index]

# Inicializar diccionario para el primer color y grosor
if current_color not in points_dict:
    points_dict[(current_color, current_thickness)] = []

last_color_change = 0  # Tiempo del último cambio de color (en segundos)
last_thickness_change = 0  # Tiempo del último cambio de grosor (en segundos)

while True:
    # Leer el frame de la cámara
    success, img = cap.read()
    if not success:
        break

    # Detectar manos
    hands, img = detector.findHands(img)

    if hands:
        hand = hands[0]  # Solo la primera mano detectada
        lmList = hand["lmList"]  # Lista de posiciones (landmarks)

        # Coordenadas del dedo índice (punta)
        x, y = lmList[8][:2]  # Índice

        # Coordenadas del pulgar (punta)
        x_thumb, y_thumb = lmList[4][:2]  # Pulgar

        # Distancia entre el índice y el pulgar
        length, _, _ = detector.findDistance(lmList[8][:2], lmList[4][:2], img)

        # Si la distancia es menor a un umbral, activa el dibujo
        if length < 50:
            if not drawing:  # Si recién empieza a dibujar
                points_dict[(current_color, current_thickness)].append(None)  # Punto de ruptura
            drawing = True
            points_dict[(current_color, current_thickness)].append((x, y))  # Agregar punto al color y grosor actuales
        else:
            drawing = False

        # Detectar gesto de señalar hacia arriba (cambio de color)
        if not drawing and time.time() - last_color_change > 5:
            if y < lmList[6][1] and y < lmList[10][1] and y < lmList[14][1] and y < lmList[18][1]:
                current_color_index = (current_color_index + 1) % len(colors)  # Cambiar al siguiente color
                current_color = colors[current_color_index]
                if (current_color, current_thickness) not in points_dict:
                    points_dict[(current_color, current_thickness)] = []  # Crear lista para nueva combinación
                last_color_change = time.time()  # Actualizar el tiempo del último cambio
                cv2.putText(img, "Color cambiado!", (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, current_color, 2)

        # Detectar gesto de puño cerrado (cambio de grosor)
        # Comprobar si todos los dedos están abajo (y más bajo es mayor en OpenCV)
        """fingers_down = all(lmList[f][1] > lmList[f - 2][1] for f in [8, 12, 16, 20])
        if not drawing and fingers_down and time.time() - last_thickness_change > 5:
            current_thickness_index = (current_thickness_index + 1) % len(thicknesses)  # Cambiar al siguiente grosor
            current_thickness = thicknesses[current_thickness_index]
            if (current_color, current_thickness) not in points_dict:
                points_dict[(current_color, current_thickness)] = []  # Crear lista para nueva combinación
            last_thickness_change = time.time()  # Actualizar el tiempo del último cambio
            cv2.putText(img, "Grosor cambiado!", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)"""

    # Dibujar los puntos en la pantalla por color y grosor
    for (color, thickness), points in points_dict.items():
        for i in range(1, len(points)):
            if points[i - 1] is None or points[i] is None:
                continue  # Saltar puntos de ruptura
            cv2.line(img, points[i - 1], points[i], color, thickness)  # Línea con color y grosor específicos

    # Mostrar el estado de "escritura"
    if drawing:
        cv2.putText(img, "Dibujando...", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 2, current_color, 3)

    # Mostrar la imagen resultante
    cv2.imshow("Dibujo", img)

    # Salir con la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [9]:
import cv2
from cvzone.HandTrackingModule import HandDetector
import time  # Para manejar el temporizador

# Configuración de la cámara
cap = cv2.VideoCapture(0)
cap.set(3, 1280)  # Ancho de la ventana
cap.set(4, 720)   # Alto de la ventana

# Detector de manos
detector = HandDetector(detectionCon=0.8, maxHands=1)

# Variables para dibujar
drawing = False  # Si está dibujando o no
points_dict = {}  # Diccionario para almacenar los puntos asociados a colores y grosores

# Lista de colores disponibles
colors = [(0, 0, 255), (0, 255, 0), (255, 0, 0), (255, 255, 0), (255, 0, 255), (0, 255, 255)]  # Rojo, verde, azul, amarillo, magenta, cian
current_color_index = 0  # Índice del color actual
current_color = colors[current_color_index]

# Grosor del dibujo
thicknesses = [5, 10, 15, 20]  # Lista de grosores disponibles
current_thickness_index = 1  # Índice del grosor actual
current_thickness = thicknesses[current_thickness_index]

# Inicializar diccionario para el primer color y grosor
if current_color not in points_dict:
    points_dict[(current_color, current_thickness)] = []

last_color_change = 0  # Tiempo del último cambio de color (en segundos)

while True:
    # Leer el frame de la cámara
    success, img = cap.read()
    if not success:
        break

    # Detectar manos
    hands, img = detector.findHands(img)

    if hands:
        hand = hands[0]  # Solo la primera mano detectada
        lmList = hand["lmList"]  # Lista de posiciones (landmarks)

        # Coordenadas del dedo índice (punta)
        x, y = lmList[8][:2]  # Índice

        # Coordenadas del pulgar (punta)
        x_thumb, y_thumb = lmList[4][:2]  # Pulgar

        # Distancia entre el índice y el pulgar
        length, _, _ = detector.findDistance(lmList[8][:2], lmList[4][:2], img)

        # Si la distancia es menor a un umbral, activa el dibujo
        if length < 50:
            if not drawing:  # Si recién empieza a dibujar
                points_dict[(current_color, current_thickness)].append(None)  # Punto de ruptura
            drawing = True
            points_dict[(current_color, current_thickness)].append((x, y))  # Agregar punto al color y grosor actuales
        else:
            drawing = False

        # Detectar gesto de señalar hacia arriba (cambio de color)
        if not drawing and time.time() - last_color_change > 5:
            if y < lmList[6][1] and y < lmList[10][1] and y < lmList[14][1] and y < lmList[18][1]:
                current_color_index = (current_color_index + 1) % len(colors)  # Cambiar al siguiente color
                current_color = colors[current_color_index]
                if (current_color, current_thickness) not in points_dict:
                    points_dict[(current_color, current_thickness)] = []  # Crear lista para nueva combinación
                last_color_change = time.time()  # Actualizar el tiempo del último cambio
                cv2.putText(img, "Color cambiado!", (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, current_color, 2)

        # Detectar gesto de puño cerrado (borrar puntos cercanos)
        fingers_down = all(
            lmList[f][1] > lmList[f - 2][1] for f in [8, 12, 16, 20]
        )  # Verificar que todos los dedos están abajo
        if fingers_down:
            cv2.putText(img, "Borrando...", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            # Limpiar puntos cercanos al índice
            for key in list(points_dict.keys()):  # Convertir en lista para evitar errores al modificar el diccionario
                points_dict[key] = [
                    point for point in points_dict[key] if point is None or (x - point[0])**2 + (y - point[1])**2 > 1600
                ]

    # Dibujar las líneas entre puntos en la pantalla
    for (color, thickness), points in points_dict.items():
        for i in range(1, len(points)):
            if points[i - 1] is None or points[i] is None:
                continue  # Saltar puntos de ruptura
            cv2.line(img, points[i - 1], points[i], color, thickness)  # Dibujar línea

    # Mostrar el estado de "escritura"
    if drawing:
        cv2.putText(img, "Dibujando...", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 2, current_color, 3)

    # Mostrar la imagen resultante
    cv2.imshow("Dibujo", img)

    # Salir con la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [21]:
import cv2
from cvzone.HandTrackingModule import HandDetector
import time  # Para manejar el temporizador

# Configuración de la cámara
cap = cv2.VideoCapture(0)
cap.set(3, 1280)  # Ancho de la ventana
cap.set(4, 720)   # Alto de la ventana

# Detector de manos
detector = HandDetector(detectionCon=0.8, maxHands=1)

# Variables para dibujar
drawing = False  # Si está dibujando o no
points_dict = {}  # Diccionario para almacenar los puntos asociados a colores y grosores

# Lista de colores disponibles
colors = [(0, 0, 255), (0, 255, 0), (255, 0, 0), (255, 255, 0), (255, 0, 255), (0, 255, 255)]  # Rojo, verde, azul, amarillo, magenta, cian
current_color_index = 0  # Índice del color actual
current_color = colors[current_color_index]

# Grosor del dibujo
thicknesses = [5, 10, 15, 20]  # Lista de grosores disponibles
current_thickness_index = 1  # Índice del grosor actual
current_thickness = thicknesses[current_thickness_index]

# Inicializar diccionario para el primer color y grosor
if current_color not in points_dict:
    points_dict[(current_color, current_thickness)] = []

last_color_change = 0  # Tiempo del último cambio de color (en segundos)


def interpolate_points(p1, p2):
    """Genera puntos entre dos extremos para una línea continua."""
    points = []
    x1, y1 = p1
    x2, y2 = p2
    distance = int(((x2 - x1)**2 + (y2 - y1)**2)**0.5)  # Distancia euclidiana
    if distance == 0:
        return [p1]  # Si los puntos son iguales, devolver solo uno
    for i in range(distance + 1):
        t = i / distance
        x = int(x1 + (x2 - x1) * t)
        y = int(y1 + (y2 - y1) * t)
        points.append((x, y))
    return points


while True:
    # Leer el frame de la cámara
    success, img = cap.read()
    if not success:
        break

    # Detectar manos
    hands, img = detector.findHands(img)

    if hands:
        hand = hands[0]  # Solo la primera mano detectada
        lmList = hand["lmList"]  # Lista de posiciones (landmarks)

        # Coordenadas del dedo índice (punta)
        x, y = lmList[8][:2]  # Índice

        # Coordenadas del pulgar (punta)
        x_thumb, y_thumb = lmList[4][:2]  # Pulgar

        # Distancia entre el índice y el pulgar
        length, _, _ = detector.findDistance(lmList[8][:2], lmList[4][:2], img)

        # Detectar si la mano está cerrada (puño) para borrar
        fingers_down = all(lmList[f][1] > lmList[f - 2][1] for f in [8, 12, 16, 20])
        if fingers_down:
            cv2.putText(img, "Borrando...", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 3)
            for (color, thickness), points in points_dict.items():
                points[:] = [point for point in points if point is None or ((x - point[0])**2 + (y - point[1])**2)**0.5 > 10]
        else:
            # Si la distancia es menor a un umbral, activa el dibujo
            if length < 50:
                if not drawing:  # Si recién empieza a dibujar
                    points_dict[(current_color, current_thickness)].append(None)  # Punto de ruptura
                drawing = True
                if len(points_dict[(current_color, current_thickness)]) > 0:
                    last_point = points_dict[(current_color, current_thickness)][-1]
                    if last_point is not None:  # Interpolar puntos entre el último y el actual
                        interpolated_points = interpolate_points(last_point, (x, y))
                        points_dict[(current_color, current_thickness)].extend(interpolated_points)
                    else:
                        points_dict[(current_color, current_thickness)].append((x, y))
                else:
                    points_dict[(current_color, current_thickness)].append((x, y))
            else:
                drawing = False

        # Detectar gesto de señalar hacia arriba (cambio de color)
        if not drawing and time.time() - last_color_change > 5:
            if y < lmList[6][1] and y < lmList[10][1] and y < lmList[14][1] and y < lmList[18][1]:
                current_color_index = (current_color_index + 1) % len(colors)  # Cambiar al siguiente color
                current_color = colors[current_color_index]
                if (current_color, current_thickness) not in points_dict:
                    points_dict[(current_color, current_thickness)] = []  # Crear lista para nueva combinación
                last_color_change = time.time()  # Actualizar el tiempo del último cambio
                cv2.putText(img, "Color cambiado!", (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, current_color, 2)

    # Dibujar las líneas entre puntos en la pantalla
    for (color, thickness), points in points_dict.items():
        for point in points:
            if point is None:
                continue  # Saltar puntos de ruptura
            cv2.circle(img, point, thickness // 2, color, -1)  # Dibujar punto sólido con el color y grosor específicos

    # Mostrar la imagen resultante
    cv2.imshow("Dibujo", img)

    # Salir con la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [20]:
import cv2
from cvzone.HandTrackingModule import HandDetector
import time  # Para manejar el temporizador

# Configuración de la cámara
cap = cv2.VideoCapture(0)
cap.set(3, 1280)  # Ancho de la ventana
cap.set(4, 720)   # Alto de la ventana

# Detector de manos
detector = HandDetector(detectionCon=0.8, maxHands=1)

# Variables para dibujar
drawing = False  # Si está dibujando o no
points_dict = {}  # Diccionario para almacenar los puntos asociados a colores y grosores

# Lista de colores disponibles
colors = [(0, 0, 255), (0, 255, 0), (255, 0, 0), (255, 255, 0), (255, 0, 255), (0, 255, 255)]  # Rojo, verde, azul, amarillo, magenta, cian
current_color_index = 0  # Índice del color actual
current_color = colors[current_color_index]

# Grosor del dibujo
thicknesses = [5, 10, 15, 20]  # Lista de grosores disponibles
current_thickness_index = 1  # Índice del grosor actual
current_thickness = thicknesses[current_thickness_index]

# Inicializar diccionario para el primer color y grosor
if current_color not in points_dict:
    points_dict[(current_color, current_thickness)] = []

last_color_change = 0  # Tiempo del último cambio de color (en segundos)


def interpolate_points(p1, p2):
    """Genera puntos entre dos extremos para una línea continua."""
    points = []
    x1, y1 = p1
    x2, y2 = p2
    distance = int(((x2 - x1)**2 + (y2 - y1)**2)**0.5)  # Distancia euclidiana
    if distance == 0:
        return [p1]  # Si los puntos son iguales, devolver solo uno
    for i in range(distance + 1):
        t = i / distance
        x = int(x1 + (x2 - x1) * t)
        y = int(y1 + (y2 - y1) * t)
        points.append((x, y))
    return points


while True:
    # Leer el frame de la cámara
    success, img = cap.read()
    if not success:
        break

    # Detectar manos
    hands, img = detector.findHands(img)

    if hands:
        hand = hands[0]  # Solo la primera mano detectada
        lmList = hand["lmList"]  # Lista de posiciones (landmarks)

        # Coordenadas del dedo índice (punta)
        x_index, y_index = lmList[8][:2]  # Índice

        # Coordenadas del pulgar (punta)
        x_thumb, y_thumb = lmList[4][:2]  # Pulgar

        x = (x_index + x_thumb) // 2
        y = (y_index + y_thumb) // 2

        # Distancia entre el índice y el pulgar
        length, _, _ = detector.findDistance(lmList[8][:2], lmList[4][:2], img)

        # Detectar si la mano está cerrada (puño) para borrar
        fingers_down = all(lmList[f][1] > lmList[f - 2][1] for f in [8, 12, 16, 20])
        if fingers_down:
            # Dibujar el círculo de borrado (punto negro con borde amarillo)
            cv2.circle(img, (x, y), current_thickness, (0, 255, 255), -1)  # Borde amarillo
            cv2.circle(img, (x, y), current_thickness - 2, (0, 0, 0), -1)  # Interior negro

            cv2.putText(img, "Borrando...", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 3)
            for (color, thickness), points in points_dict.items():
                points[:] = [point for point in points if point is None or ((x - point[0])**2 + (y - point[1])**2)**0.5 > 10]
        else:
            # Si la distancia es menor a un umbral, activa el dibujo
            if length < 50:
                if not drawing:  # Si recién empieza a dibujar
                    points_dict[(current_color, current_thickness)].append(None)  # Punto de ruptura
                drawing = True
                if len(points_dict[(current_color, current_thickness)]) > 0:
                    last_point = points_dict[(current_color, current_thickness)][-1]
                    if last_point is not None:  # Interpolar puntos entre el último y el actual
                        interpolated_points = interpolate_points(last_point, (x, y))
                        points_dict[(current_color, current_thickness)].extend(interpolated_points)
                    else:
                        points_dict[(current_color, current_thickness)].append((x, y))
                else:
                    points_dict[(current_color, current_thickness)].append((x, y))
            else:
                drawing = False

        # Detectar gesto de señalar hacia arriba (cambio de color)
        if not drawing and time.time() - last_color_change > 5:
            if y < lmList[6][1] and y < lmList[10][1] and y < lmList[14][1] and y < lmList[18][1]:
                current_color_index = (current_color_index + 1) % len(colors)  # Cambiar al siguiente color
                current_color = colors[current_color_index]
                if (current_color, current_thickness) not in points_dict:
                    points_dict[(current_color, current_thickness)] = []  # Crear lista para nueva combinación
                last_color_change = time.time()  # Actualizar el tiempo del último cambio
                cv2.putText(img, "Color cambiado!", (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, current_color, 2)

    # Dibujar las líneas entre puntos en la pantalla
    for (color, thickness), points in points_dict.items():
        for point in points:
            if point is None:
                continue  # Saltar puntos de ruptura
            cv2.circle(img, point, thickness // 2, color, -1)  # Dibujar punto sólido con el color y grosor específicos

    # Mostrar la imagen resultante
    cv2.imshow("Dibujo", img)

    # Salir con la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [22]:
import cv2
from cvzone.HandTrackingModule import HandDetector
import time  # Para manejar el temporizador

# Configuración de la cámara
cap = cv2.VideoCapture(0)
cap.set(3, 1280)  # Ancho de la ventana
cap.set(4, 720)   # Alto de la ventana

# Detector de manos
detector = HandDetector(detectionCon=0.8, maxHands=1)

# Variables para dibujar
drawing = False  # Si está dibujando o no
points_dict = {}  # Diccionario para almacenar los puntos asociados a colores y grosores

# Lista de colores disponibles
colors = [(0, 0, 255), (0, 255, 0), (255, 0, 0), (255, 255, 0), (255, 0, 255), (0, 255, 255)]  # Rojo, verde, azul, amarillo, magenta, cian
current_color_index = 0  # Índice del color actual
current_color = colors[current_color_index]

# Grosor del dibujo
thicknesses = [5, 10, 15, 20]  # Lista de grosores disponibles
current_thickness_index = 1  # Índice del grosor actual
current_thickness = thicknesses[current_thickness_index]

# Inicializar diccionario para el primer color y grosor
if current_color not in points_dict:
    points_dict[(current_color, current_thickness)] = []

last_color_change = 0  # Tiempo del último cambio de color (en segundos)


def interpolate_points(p1, p2):
    """Genera puntos entre dos extremos para una línea continua."""
    points = []
    x1, y1 = p1
    x2, y2 = p2
    distance = int(((x2 - x1)**2 + (y2 - y1)**2)**0.5)  # Distancia euclidiana
    if distance == 0:
        return [p1]  # Si los puntos son iguales, devolver solo uno
    for i in range(distance + 1):
        t = i / distance
        x = int(x1 + (x2 - x1) * t)
        y = int(y1 + (y2 - y1) * t)
        points.append((x, y))
    return points


def is_index_up(lmList):
    """Detectar si el dedo índice está levantado."""
    return lmList[8][1] < lmList[6][1] and lmList[8][1] < lmList[10][1] and lmList[8][1] < lmList[14][1] and lmList[8][1] < lmList[18][1]

while True:
    # Leer el frame de la cámara
    success, img = cap.read()
    if not success:
        break

    # Detectar manos
    hands, img = detector.findHands(img)

    if hands:
        hand = hands[0]  # Solo la primera mano detectada
        lmList = hand["lmList"]  # Lista de posiciones (landmarks)

        # Coordenadas del dedo índice (punta)
        x_index, y_index = lmList[8][:2]  # Índice

        # Coordenadas del pulgar (punta)
        x_thumb, y_thumb = lmList[4][:2]  # Pulgar

        x = (x_index + x_thumb) // 2
        y = (y_index + y_thumb) // 2

        # Distancia entre el índice y el pulgar
        length, _, _ = detector.findDistance(lmList[8][:2], lmList[4][:2], img)

        # Detectar si la mano está cerrada (puño) para borrar
        fingers_down = all(lmList[f][1] > lmList[f - 2][1] for f in [8, 12, 16, 20])
        if fingers_down:
            # Dibujar el círculo de borrado (punto negro con borde amarillo)
            cv2.circle(img, (x, y), current_thickness, (0, 255, 255), -1)  # Borde amarillo
            cv2.circle(img, (x, y), current_thickness - 2, (0, 0, 0), -1)  # Interior negro

            cv2.putText(img, "Borrando...", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 3)
            for (color, thickness), points in points_dict.items():
                points[:] = [point for point in points if point is None or ((x - point[0])**2 + (y - point[1])**2)**0.5 > 10]
        else:
            # Si la distancia es menor a un umbral, activa el dibujo
            if length < 50:
                if not drawing:  # Si recién empieza a dibujar
                    points_dict[(current_color, current_thickness)].append(None)  # Punto de ruptura
                drawing = True
                if len(points_dict[(current_color, current_thickness)]) > 0:
                    last_point = points_dict[(current_color, current_thickness)][-1]
                    if last_point is not None:  # Interpolar puntos entre el último y el actual
                        interpolated_points = interpolate_points(last_point, (x, y))
                        points_dict[(current_color, current_thickness)].extend(interpolated_points)
                    else:
                        points_dict[(current_color, current_thickness)].append((x, y))
                else:
                    points_dict[(current_color, current_thickness)].append((x, y))
            else:
                drawing = False

        # Si el índice está levantado, mostrar la interfaz de selección de color
        if is_index_up(lmList):
            # Dibujar los rectángulos de color en la pantalla
            color_rectangles = []
            for i, color in enumerate(colors):
                x1, y1 = 50, 50 + i * 50  # Posición de cada color en la interfaz
                cv2.rectangle(img, (x1, y1), (x1 + 40, y1 + 40), color, -1)
                color_rectangles.append((color, (x1, y1, 40, 40)))  # Guardamos las coordenadas para comparación

            # Detectar si se hace clic sobre uno de los colores
            if length < 50:  # Si la distancia es corta, permitimos seleccionar el color
                for color, (x1, y1, w, h) in color_rectangles:
                    if x1 <= x <= x1 + w and y1 <= y <= y1 + h:
                        current_color = color
                        cv2.putText(img, "Color seleccionado!", (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, current_color, 2)
                        break

            # Desactivar el dibujo mientras se selecciona el color
            drawing = False

    # Dibujar las líneas entre puntos en la pantalla
    for (color, thickness), points in points_dict.items():
        for point in points:
            if point is None:
                continue  # Saltar puntos de ruptura
            cv2.circle(img, point, thickness // 2, color, -1)  # Dibujar punto sólido con el color y grosor específicos

    # Mostrar la imagen resultante
    cv2.imshow("Dibujo", img)

    # Salir con la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [4]:
import cv2
from cvzone.HandTrackingModule import HandDetector
import time  # Para manejar el temporizador

# Configuración de la cámara
cap = cv2.VideoCapture(0)
cap.set(3, 1280)  # Ancho de la ventana
cap.set(4, 720)   # Alto de la ventana

# Detector de manos
detector = HandDetector(detectionCon=0.8, maxHands=1)

# Variables para dibujar
drawing = False  # Si está dibujando o no
points_dict = {}  # Diccionario para almacenar los puntos asociados a colores y grosores

# Lista de colores disponibles
colors = [(0, 0, 255), (0, 255, 0), (255, 0, 0), (255, 255, 0), (255, 0, 255), (0, 255, 255)]  # Rojo, verde, azul, amarillo, magenta, cian
current_color_index = 0  # Índice del color actual
current_color = colors[current_color_index]

# Grosor del dibujo
thicknesses = [5, 10, 15, 20]  # Lista de grosores disponibles
current_thickness_index = 1  # Índice del grosor actual
current_thickness = thicknesses[current_thickness_index]

# Inicializar diccionario para el primer color y grosor
if (current_color, current_thickness) not in points_dict:
    points_dict[(current_color, current_thickness)] = []  # Asegurarse de que la clave exista

last_color_change = 0  # Tiempo del último cambio de color (en segundos)
interface_active = False  # La interfaz comienza activa

# Función para interpolar los puntos y dibujar líneas continuas
def interpolate_points(p1, p2):
    points = []
    x1, y1 = p1
    x2, y2 = p2
    distance = int(((x2 - x1)**2 + (y2 - y1)**2)**0.5)
    if distance == 0:
        return [p1]  # Si los puntos son iguales, devolver solo uno
    for i in range(distance + 1):
        t = i / distance
        x = int(x1 + (x2 - x1) * t)
        y = int(y1 + (y2 - y1) * t)
        points.append((x, y))
    return points

# Función para verificar si el índice está levantado
def is_index_up(lmList):
    return lmList[8][1] < lmList[6][1] and lmList[8][1] < lmList[10][1] and lmList[8][1] < lmList[14][1] and lmList[8][1] < lmList[18][1]

while True:
    # Leer el frame de la cámara
    success, img = cap.read()
    if not success:
        break

    # Detectar manos
    hands, img = detector.findHands(img)

    if hands:
        hand = hands[0]  # Solo la primera mano detectada
        lmList = hand["lmList"]  # Lista de posiciones (landmarks)

        # Coordenadas del dedo índice (punta)
        x_index, y_index = lmList[8][:2]  # Índice

        # Coordenadas del pulgar (punta)
        x_thumb, y_thumb = lmList[4][:2]  # Pulgar

        x = (x_index + x_thumb) // 2
        y = (y_index + y_thumb) // 2

        # Distancia entre el índice y el pulgar
        length, _, _ = detector.findDistance(lmList[8][:2], lmList[4][:2], img)

        # Detectar si la mano está cerrada (puño) para borrar
        fingers_down = all(lmList[f][1] > lmList[f - 2][1] for f in [8, 12, 16, 20])
        if fingers_down and not interface_active:  # Solo permitir borrar si la interfaz no está activa
            # Dibujar el círculo de borrado (punto negro con borde amarillo)
            cv2.circle(img, (x, y), current_thickness, (0, 255, 255), -1)  # Borde amarillo
            cv2.circle(img, (x, y), current_thickness - 2, (0, 0, 0), -1)  # Interior negro

            cv2.putText(img, "Borrando...", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 3)
            for (color, thickness), points in points_dict.items():
                points[:] = [point for point in points if point is None or ((x - point[0])**2 + (y - point[1])**2)**0.5 > 10]


        elif not interface_active & is_index_up(lmList):
           interface_active = True   
            
        elif not interface_active:  # Solo permitir dibujar si la interfaz no está activa
            # Si la distancia es menor a un umbral, activa el dibujo
            if length < 50:
                if not drawing:  # Si recién empieza a dibujar
                    points_dict[(current_color, current_thickness)].append(None)  # Punto de ruptura
                drawing = True
                if len(points_dict[(current_color, current_thickness)]) > 0:
                    last_point = points_dict[(current_color, current_thickness)][-1]
                    if last_point is not None:  # Interpolar puntos entre el último y el actual
                        interpolated_points = interpolate_points(last_point, (x, y))
                        points_dict[(current_color, current_thickness)].extend(interpolated_points)
                    else:
                        points_dict[(current_color, current_thickness)].append((x, y))
                else:
                    points_dict[(current_color, current_thickness)].append((x, y))
            else:
                drawing = False

        # Si la interfaz está activa, dibujar la selección de colores
        if interface_active:
            # Dibujar los rectángulos de color en la parte inferior central
            color_rectangles = []
            num_colors = len(colors)
            rect_width = 60
            spacing = 20
            start_x = (img.shape[1] - (num_colors * rect_width + (num_colors - 1) * spacing)) // 2  # Centrar horizontalmente
            start_y = img.shape[0] - 100  # Ubicar en la parte inferior

            for i, color in enumerate(colors):
                x1, y1 = start_x + i * (rect_width + spacing), start_y
                cv2.rectangle(img, (x1, y1), (x1 + rect_width, y1 + 50), color, -1)
                color_rectangles.append((color, (x1, y1, rect_width, 50)))  # Guardamos las coordenadas para comparación

            # Detectar si se hace clic sobre uno de los colores
            if length < 50:  # Si la distancia es corta, permitimos seleccionar el color
                for color, (x1, y1, w, h) in color_rectangles:
                    if x1 <= x <= x1 + w and y1 <= y <= y1 + h:
                        current_color = color
                        cv2.putText(img, "Color seleccionado!", (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, current_color, 2)
                        interface_active = False  # Desactivar la interfaz después de seleccionar un color
                        break

    # Dibujar las líneas entre puntos en la pantalla
    for (color, thickness), points in points_dict.items():
        for point in points:
            if point is None:
                continue  # Saltar puntos de ruptura
            cv2.circle(img, point, thickness // 2, color, -1)  # Dibujar punto sólido con el color y grosor específicos

    # Mostrar la imagen resultante
    cv2.imshow("Dibujo", img)

    # Salir con la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [1]:
import cv2
from cvzone.HandTrackingModule import HandDetector
import time  # Para manejar el temporizador

# Configuración de la cámara
cap = cv2.VideoCapture(0)
cap.set(3, 1280)  # Ancho de la ventana
cap.set(4, 720)   # Alto de la ventana

# Detector de manos
detector = HandDetector(detectionCon=0.8, maxHands=1)

# Variables para dibujar
drawing = False  # Si está dibujando o no
points_dict = {}  # Diccionario para almacenar los puntos asociados a colores y grosores

# Lista de colores disponibles
colors = [(0, 0, 255), (0, 255, 0), (255, 0, 0), (255, 255, 0), (255, 0, 255), (0, 255, 255)]  # Rojo, verde, azul, amarillo, magenta, cian
current_color_index = 0  # Índice del color actual
current_color = colors[current_color_index]

# Grosor del dibujo
thicknesses = [5, 10, 15, 20]  # Lista de grosores disponibles
current_thickness_index = 1  # Índice del grosor actual
current_thickness = thicknesses[current_thickness_index]

# Inicializar diccionario para el primer color y grosor
if (current_color, current_thickness) not in points_dict:
    points_dict[(current_color, current_thickness)] = []  # Asegurarse de que la clave exista

interface_active = False  # La interfaz comienza inactiva

# Función para interpolar los puntos y dibujar líneas continuas
def interpolate_points(p1, p2):
    points = []
    x1, y1 = p1
    x2, y2 = p2
    distance = int(((x2 - x1)**2 + (y2 - y1)**2)**0.5)
    if distance == 0:
        return [p1]  # Si los puntos son iguales, devolver solo uno
    for i in range(1,distance + 1):
        t = i / distance
        x = int(x1 + (x2 - x1) * t)
        y = int(y1 + (y2 - y1) * t)
        if (x,y) not in points:
            points.append((x, y))
    return points

while True:
    # Leer el frame de la cámara
    success, img = cap.read()
    if not success:
        break

    # Detectar manos
    hands, img = detector.findHands(img)

    if hands:
        hand = hands[0]  # Solo la primera mano detectada
        lmList = hand["lmList"]  # Lista de posiciones (landmarks)

        # Coordenadas del dedo índice (punta)
        x_index, y_index = lmList[8][:2]  # Índice

        # Coordenadas del pulgar (punta)
        x_thumb, y_thumb = lmList[4][:2]  # Pulgar

        x = (x_index + x_thumb) // 2
        y = (y_index + y_thumb) // 2

        # Distancia entre el índice y el pulgar
        length, _, _ = detector.findDistance(lmList[8][:2], lmList[4][:2], img)

        # Detectar si la mano está cerrada (puño) para borrar
        fingers_down = all(lmList[f][1] > lmList[f - 2][1] for f in [8, 12, 16, 20])
        if fingers_down and not interface_active:  # Solo permitir borrar si la interfaz no está activa
            # Dibujar el círculo de borrado
            cv2.circle(img, (x, y), current_thickness, (0, 255, 255), -1)  # Borde amarillo
            cv2.circle(img, (x, y), current_thickness - 2, (0, 0, 0), -1)  # Interior negro
            for (color, thickness), points in points_dict.items():
                points[:] = [point for point in points if point is None or ((x - point[0])**2 + (y - point[1])**2)**0.5 > 10]

        elif not interface_active and lmList[8][1] < lmList[6][1]:  # Activar interfaz si el índice está levantado
            interface_active = True
        
        elif not interface_active:  # Solo permitir dibujar si la interfaz no está activa
            # Si la distancia es menor a un umbral, activa el dibujo
            if length < 50:
                if (current_color, current_thickness) not in points_dict:
                    points_dict[(current_color, current_thickness)] = []  # Asegurarse de que la clave exista
                if not drawing:  # Si recién empieza a dibujar
                    points_dict[(current_color, current_thickness)].append(None)  # Punto de ruptura
                drawing = True
                if len(points_dict[(current_color, current_thickness)]) > 0:
                    last_point = points_dict[(current_color, current_thickness)][-1]
                    if last_point is not None:  # Interpolar puntos entre el último y el actual
                        interpolated_points = interpolate_points(last_point, (x, y))
                        points_dict[(current_color, current_thickness)].extend(interpolated_points)
                    else:
                        if (x,y) not in points_dict[(current_color, current_thickness)]:
                            points_dict[(current_color, current_thickness)].append((x, y))
                else:
                    if (x,y) not in points_dict[(current_color, current_thickness)]:
                        points_dict[(current_color, current_thickness)].append((x, y))
            else:
                drawing = False

        # Si la interfaz está activa, dibujar la selección de colores
        if interface_active:
            # Dibujar los rectángulos de color en la parte inferior central
            color_rectangles = []
            num_colors = len(colors)
            rect_width = 60
            spacing = 20
            start_x = (img.shape[1] - (num_colors * rect_width + (num_colors - 1) * spacing)) // 2
            start_y = img.shape[0] - 100

            for i, color in enumerate(colors):
                x1, y1 = start_x + i * (rect_width + spacing), start_y
                cv2.rectangle(img, (x1, y1), (x1 + rect_width, y1 + 50), color, -1)
                color_rectangles.append((color, (x1, y1, rect_width, 50)))

            # Detectar si se hace clic sobre uno de los colores
            if length < 50:  # Si la distancia es corta, permitimos seleccionar el color
                for color, (x1, y1, w, h) in color_rectangles:
                    if x1 <= x <= x1 + w and y1 <= y <= y1 + h:
                        current_color = color
                        current_thickness = 10
                        interface_active = False  # Desactivar la interfaz después de seleccionar un color
                        drawing = False
                        break

    # Dibujar las líneas entre puntos en la pantalla
    for (color, thickness), points in points_dict.items():
        for point in points:
            if point is None:
                continue  # Saltar puntos de ruptura
            cv2.circle(img, point, thickness // 2, color, -1)  # Dibujar punto sólido con el color y grosor específicos

    # Mostrar la imagen resultante
    cv2.imshow("Dibujo", img)

    # Salir con la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [1]:
import cv2
from cvzone.HandTrackingModule import HandDetector

# Configuración de la cámara
cap = cv2.VideoCapture(0)
cap.set(3, 1280)  # Ancho de la ventana
cap.set(4, 720)   # Alto de la ventana

# Detector de manos
detector = HandDetector(detectionCon=0.8, maxHands=1)

# Variables para dibujar
drawing = False  # Si está dibujando o no
points_dict = {}  # Diccionario para almacenar los puntos asociados a colores, grosores y modos de pintura

# Lista de colores y grosores disponibles
colors = [(0, 0, 255), (0, 255, 0), (255, 0, 0), (255, 255, 0), (255, 0, 255), (0, 255, 255)]  # Rojo, verde, azul, amarillo, magenta, cian
thicknesses = [5, 10, 15, 20]  # Lista de grosores disponibles
paint_modes = ['normal', 'highlighter', 'oil']  # Modos de pintura
current_color = colors[0]
current_thickness = thicknesses[1]
current_paint_mode = paint_modes[0]

# Inicializar diccionario para el primer color, grosor y modo de pintura
if (current_color, current_thickness, current_paint_mode) not in points_dict:
    points_dict[(current_color, current_thickness, current_paint_mode)] = []

# Banderas para las interfaces
color_interface_active = False
thickness_interface_active = False
paint_mode_interface_active = False

# Función para interpolar los puntos y dibujar líneas continuas
def interpolate_points(p1, p2):
    points = []
    x1, y1 = p1
    x2, y2 = p2
    distance = int(((x2 - x1)**2 + (y2 - y1)**2)**0.5)
    if distance == 0:
        return [p1]
    for i in range(1, distance + 1):
        t = i / distance
        x = int(x1 + (x2 - x1) * t)
        y = int(y1 + (y2 - y1) * t)
        if (x, y) not in points:
            points.append((x, y))
    return points

# Función para manejar los diferentes modos de pintura
def draw_paint_mode(img, points, mode, thickness, color):
    if mode == 'normal':
        for point in points:
            if point is None:
                continue
            cv2.circle(img, point, thickness // 2, color, -1)
    elif mode == 'highlighter':
        for point in points:
            if point is None:
                continue
            cv2.circle(img, point, thickness // 2, (255, 255, 0), -1)  # Amarillo para subrayador
    elif mode == 'oil':
        for point in points:
            if point is None:
                continue
            cv2.circle(img, point, thickness // 2, (0, 255, 255), -1)  # Color para pintura al óleo

# Bucle principal
while True:
    # Leer el frame de la cámara
    success, img = cap.read()
    if not success:
        break

    # Detectar manos
    hands, img = detector.findHands(img)

    # Dibujar las líneas entre puntos en la pantalla
    for (color, thickness, mode), points in points_dict.items():
        draw_paint_mode(img, points, mode, thickness, color)

    if hands:
        hand = hands[0]  # Solo la primera mano detectada
        lmList = hand["lmList"]  # Lista de posiciones (landmarks)

        # Coordenadas del dedo índice (punta)
        x_index, y_index = lmList[8][:2]  # Índice

        # Coordenadas del pulgar (punta)
        x_thumb, y_thumb = lmList[4][:2]  # Pulgar

        x = (x_index + x_thumb) // 2
        y = (y_index + y_thumb) // 2

        # Distancia entre el índice y el pulgar
        length, _, _ = detector.findDistance(lmList[8][:2], lmList[4][:2], img)

        # Detectar si la mano está cerrada (puño) para borrar
        fingers_down = all(lmList[f][1] > lmList[f - 2][1] for f in [8, 12, 16, 20])
        if fingers_down and not (color_interface_active or thickness_interface_active or paint_mode_interface_active):
            # Dibujar el círculo de borrado
            cv2.circle(img, (x, y), current_thickness, (0, 255, 255), -1)  # Borde amarillo
            cv2.circle(img, (x, y), current_thickness - 2, (0, 0, 0), -1)  # Interior negro
            for (color, thickness, mode), points in points_dict.items():
                points[:] = [point for point in points if point is None or ((x - point[0])**2 + (y - point[1])**2)**0.5 > 10]

        elif not color_interface_active and not thickness_interface_active and not paint_mode_interface_active and lmList[8][1] < lmList[6][1]:
            # Activar interfaz de color, grosor o modo de pintura según posición del dedo
            if x_index < img.shape[1] // 3:
                color_interface_active = True
            elif x_index < img.shape[1] * 2 // 3:
                thickness_interface_active = True
            else:
                paint_mode_interface_active = True

        elif not (color_interface_active or thickness_interface_active or paint_mode_interface_active):  # Solo permitir dibujar si ninguna interfaz está activa
            if length < 50:
                if (current_color, current_thickness, current_paint_mode) not in points_dict:
                    points_dict[(current_color, current_thickness, current_paint_mode)] = []
                if not drawing:
                    points_dict[(current_color, current_thickness, current_paint_mode)].append(None)
                drawing = True
                if len(points_dict[(current_color, current_thickness, current_paint_mode)]) > 0:
                    last_point = points_dict[(current_color, current_thickness, current_paint_mode)][-1]
                    if last_point is not None:
                        interpolated_points = interpolate_points(last_point, (x, y))
                        points_dict[(current_color, current_thickness, current_paint_mode)].extend(interpolated_points)
                    else:
                        points_dict[(current_color, current_thickness, current_paint_mode)].append((x, y))
            else:
                drawing = False

        # Interfaz de selección de color
        if color_interface_active:
            color_rectangles = []
            num_colors = len(colors)
            rect_width = 60
            spacing = 20
            start_x = (img.shape[1] - (num_colors * rect_width + (num_colors - 1) * spacing)) // 2
            start_y = img.shape[0] - 100

            for i, color in enumerate(colors):
                x1, y1 = start_x + i * (rect_width + spacing), start_y
                cv2.rectangle(img, (x1, y1), (x1 + rect_width, y1 + 50), color, -1)
                color_rectangles.append((color, (x1, y1, rect_width, 50)))

            if length < 50:
                for color, (x1, y1, w, h) in color_rectangles:
                    if x1 <= x <= x1 + w and y1 <= y <= y1 + h:
                        current_color = color
                        color_interface_active = False
                        break

        # Interfaz de selección de grosor
        if thickness_interface_active:
            thickness_circles = []
            num_thicknesses = len(thicknesses)
            spacing = 20  # Espacio entre los círculos
            start_x = (img.shape[1] - (num_thicknesses * (max(thicknesses) * 2 + spacing) - spacing)) // 2
            start_y = img.shape[0] - 200

            for i, thickness in enumerate(thicknesses):
                center_x = start_x + i * (thickness * 2 + spacing)  # Ajuste de posición según el grosor
                center_y = start_y
                # Dibujar un círculo con borde negro y relleno con el color actual
                cv2.circle(img, (center_x, center_y), thickness, (0, 0, 0), -1)  # Borde negro
                cv2.circle(img, (center_x, center_y), thickness - 1, current_color, -1)  # Relleno con el color actual
                thickness_circles.append((thickness, (center_x, center_y, thickness)))

            # Detectar si el clic está dentro de algún círculo
            if length < 50:
                for thickness, (center_x, center_y, radius) in thickness_circles:
                    if (x - center_x) ** 2 + (y - center_y) ** 2 <= radius ** 2:  # Verificar si el clic está dentro del círculo
                        current_thickness = thickness
                        thickness_interface_active = False
                        break

        # Interfaz de selección de modo de pintura
        if paint_mode_interface_active:
            mode_rectangles = []
            num_modes = len(paint_modes)
            rect_width = 60
            spacing = 20
            start_x = (img.shape[1] - (num_modes * rect_width + (num_modes - 1) * spacing)) // 2
            start_y = img.shape[0] - 300

            for i, mode in enumerate(paint_modes):
                x1, y1 = start_x + i * (rect_width + spacing), start_y
                cv2.rectangle(img, (x1, y1), (x1 + rect_width, y1 + 50), (200, 200, 200), -1)
                cv2.putText(img, f"{mode}", (x1 + 10, y1 + 35), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
                mode_rectangles.append((mode, (x1, y1, rect_width, 50)))

            if length < 50:
                for mode, (x1, y1, w, h) in mode_rectangles:
                    if x1 <= x <= x1 + w and y1 <= y <= y1 + h:
                        current_paint_mode = mode
                        paint_mode_interface_active = False
                        break

    # Mostrar la imagen resultante
    cv2.imshow("Dibujo", img)

    # Salir con la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [3]:
import cv2
from cvzone.HandTrackingModule import HandDetector

# Configuración de la cámara
cap = cv2.VideoCapture(0)
cap.set(3, 1280)  # Ancho de la ventana
cap.set(4, 720)   # Alto de la ventana

# Detector de manos
detector = HandDetector(detectionCon=0.8, maxHands=1)

# Variables para dibujar
drawing = False  # Si está dibujando o no
points_dict = {}  # Diccionario para almacenar los puntos asociados a colores, grosores y modos de pintura

# Lista de colores y grosores disponibles
colors = [(0, 0, 255), (0, 255, 0), (255, 0, 0), (255, 255, 0), (255, 0, 255), (0, 255, 255)]  # Rojo, verde, azul, amarillo, magenta, cian
thicknesses = [5, 10, 15, 20]  # Lista de grosores disponibles
paint_modes = ['normal', 'highlighter', 'oil']  # Modos de pintura
current_color = colors[0]
current_thickness = thicknesses[1]
current_paint_mode = paint_modes[0]

# Inicializar diccionario para el primer color, grosor y modo de pintura
if (current_color, current_thickness, current_paint_mode) not in points_dict:
    points_dict[(current_color, current_thickness, current_paint_mode)] = []

# Banderas para las interfaces
color_interface_active = False
thickness_interface_active = False
paint_mode_interface_active = False

# Función para interpolar los puntos y dibujar líneas continuas
def interpolate_points(p1, p2):
    points = []
    x1, y1 = p1
    x2, y2 = p2
    distance = int(((x2 - x1)**2 + (y2 - y1)**2)**0.5)
    if distance == 0:
        return [p1]
    for i in range(1, distance + 1):
        t = i / distance
        x = int(x1 + (x2 - x1) * t)
        y = int(y1 + (y2 - y1) * t)
        if (x, y) not in points:
            points.append((x, y))
    return points

"""# Función para manejar los diferentes modos de pintura
def draw_paint_mode(img, points, mode, thickness, color):
    if mode == 'normal':
        for point in points:
            if point is None:
                continue
            cv2.circle(img, point, thickness // 2, color, -1)
    elif mode == 'highlighter':
        for point in points:
            if point is None:
                continue
            cv2.circle(img, point, thickness // 2, (255, 255, 0), -1)  # Amarillo para subrayador
    elif mode == 'oil':
        for point in points:
            if point is None:
                continue
            cv2.circle(img, point, thickness // 2, (0, 255, 255), -1)  # Color para pintura al óleo"""

def draw_paint_mode(img, points, mode, thickness, color):
    if mode == 'normal':
        for point in points:
            if point is None:
                continue
            cv2.circle(img, point, thickness // 2, color, -1)

    elif mode == 'highlighter':
        overlay = img.copy()  # Crear una capa de superposición
        for point in points:
            if point is None:
                continue
            # Dibujar el subrayador con transparencia
            cv2.circle(overlay, point, thickness // 2, color, -1)
        # Combinar con el frame original
        alpha = 0.4  # Nivel de transparencia
        cv2.addWeighted(overlay, alpha, img, 1 - alpha, 0, img)

    elif mode == 'oil':
        for point in points:
            if point is None:
                continue
            # Dibujar el círculo base
            cv2.circle(img, point, thickness // 2, color, -1)
            # Simular un efecto de óleo con contorno suave
            cv2.circle(img, point, thickness, color, 1)  # Contorno externo suave


# Bucle principal
while True:
    # Leer el frame de la cámara
    success, img = cap.read()
    if not success:
        break

    # Detectar manos
    hands, img = detector.findHands(img)

    # Dibujar las líneas entre puntos en la pantalla
    for (color, thickness, mode), points in points_dict.items():
        draw_paint_mode(img, points, mode, thickness, color)

    if hands:
        hand = hands[0]  # Solo la primera mano detectada
        lmList = hand["lmList"]  # Lista de posiciones (landmarks)

        # Coordenadas del dedo índice (punta)
        x_index, y_index = lmList[8][:2]  # Índice

        # Coordenadas del pulgar (punta)
        x_thumb, y_thumb = lmList[4][:2]  # Pulgar

        x = (x_index + x_thumb) // 2
        y = (y_index + y_thumb) // 2

        # Distancia entre el índice y el pulgar
        length, _, _ = detector.findDistance(lmList[8][:2], lmList[4][:2], img)

        # Detectar si la mano está cerrada (puño) para borrar
        fingers_down = all(lmList[f][1] > lmList[f - 2][1] for f in [8, 12, 16, 20])
        if fingers_down and not (color_interface_active or thickness_interface_active or paint_mode_interface_active):
            # Dibujar el círculo de borrado
            cv2.circle(img, (x, y), current_thickness, (0, 255, 255), -1)  # Borde amarillo
            cv2.circle(img, (x, y), current_thickness - 2, (0, 0, 0), -1)  # Interior negro
            for (color, thickness, mode), points in points_dict.items():
                points[:] = [point for point in points if point is None or ((x - point[0])**2 + (y - point[1])**2)**0.5 > 10]

        elif not color_interface_active and not thickness_interface_active and not paint_mode_interface_active and lmList[8][1] < lmList[6][1]:
            # Activar interfaz de color, grosor o modo de pintura según posición del dedo
            if x_index < img.shape[1] // 3:
                color_interface_active = True
            elif x_index < img.shape[1] * 2 // 3:
                thickness_interface_active = True
            else:
                paint_mode_interface_active = True

        elif not (color_interface_active or thickness_interface_active or paint_mode_interface_active):  # Solo permitir dibujar si ninguna interfaz está activa
            if length < 50:
                if (current_color, current_thickness, current_paint_mode) not in points_dict:
                    points_dict[(current_color, current_thickness, current_paint_mode)] = []
                if not drawing:
                    points_dict[(current_color, current_thickness, current_paint_mode)].append(None)
                drawing = True
                if len(points_dict[(current_color, current_thickness, current_paint_mode)]) > 0:
                    last_point = points_dict[(current_color, current_thickness, current_paint_mode)][-1]
                    if last_point is not None:
                        interpolated_points = interpolate_points(last_point, (x, y))
                        points_dict[(current_color, current_thickness, current_paint_mode)].extend(interpolated_points)
                    else:
                        points_dict[(current_color, current_thickness, current_paint_mode)].append((x, y))
            else:
                drawing = False

        # Interfaz de selección de color
        if color_interface_active:
            color_rectangles = []
            num_colors = len(colors)
            rect_width = 60
            spacing = 20
            start_x = (img.shape[1] - (num_colors * rect_width + (num_colors - 1) * spacing)) // 2
            start_y = img.shape[0] - 100

            for i, color in enumerate(colors):
                x1, y1 = start_x + i * (rect_width + spacing), start_y
                cv2.rectangle(img, (x1, y1), (x1 + rect_width, y1 + 50), color, -1)
                color_rectangles.append((color, (x1, y1, rect_width, 50)))

            if length < 50:
                for color, (x1, y1, w, h) in color_rectangles:
                    if x1 <= x <= x1 + w and y1 <= y <= y1 + h:
                        current_color = color
                        color_interface_active = False
                        break

        # Interfaz de selección de grosor
        if thickness_interface_active:
            thickness_circles = []
            num_thicknesses = len(thicknesses)
            spacing = 20  # Espacio entre los círculos
            start_x = (img.shape[1] - (num_thicknesses * (max(thicknesses) * 2 + spacing) - spacing)) // 2
            start_y = img.shape[0] - 200

            for i, thickness in enumerate(thicknesses):
                center_x = start_x + i * (thickness * 2 + spacing)  # Ajuste de posición según el grosor
                center_y = start_y
                # Dibujar un círculo con borde negro y relleno con el color actual
                cv2.circle(img, (center_x, center_y), thickness, (0, 0, 0), -1)  # Borde negro
                cv2.circle(img, (center_x, center_y), thickness - 1, current_color, -1)  # Relleno con el color actual
                thickness_circles.append((thickness, (center_x, center_y, thickness)))

            # Detectar si el clic está dentro de algún círculo
            if length < 50:
                for thickness, (center_x, center_y, radius) in thickness_circles:
                    if (x - center_x) ** 2 + (y - center_y) ** 2 <= radius ** 2:  # Verificar si el clic está dentro del círculo
                        current_thickness = thickness
                        thickness_interface_active = False
                        break

        # Interfaz de selección de modo de pintura
        if paint_mode_interface_active:
            mode_rectangles = []
            num_modes = len(paint_modes)
            rect_width = 60
            spacing = 20
            start_x = (img.shape[1] - (num_modes * rect_width + (num_modes - 1) * spacing)) // 2
            start_y = img.shape[0] - 300

            for i, mode in enumerate(paint_modes):
                x1, y1 = start_x + i * (rect_width + spacing), start_y
                cv2.rectangle(img, (x1, y1), (x1 + rect_width, y1 + 50), (200, 200, 200), -1)
                cv2.putText(img, f"{mode}", (x1 + 10, y1 + 35), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
                mode_rectangles.append((mode, (x1, y1, rect_width, 50)))

            if length < 50:
                for mode, (x1, y1, w, h) in mode_rectangles:
                    if x1 <= x <= x1 + w and y1 <= y <= y1 + h:
                        current_paint_mode = mode
                        paint_mode_interface_active = False
                        break

    # Mostrar la imagen resultante
    cv2.imshow("Dibujo", img)

    # Salir con la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [7]:
import cv2
import numpy as np
from cvzone.HandTrackingModule import HandDetector

# Configuración de la cámara
cap = cv2.VideoCapture(0)
cap.set(3, 1280)  # Ancho de la ventana
cap.set(4, 720)   # Alto de la ventana

# Detector de manos
detector = HandDetector(detectionCon=0.8, maxHands=1)

# Variables para dibujar
drawing = False  # Si está dibujando o no
points_dict = {}  # Diccionario para almacenar los puntos asociados a colores, grosores y modos de pintura

# Lista de colores y grosores disponibles
colors = [(0, 0, 255), (0, 255, 0), (255, 0, 0), (255, 255, 0), (255, 0, 255), (0, 255, 255)]  # Rojo, verde, azul, amarillo, magenta, cian
thicknesses = [5, 10, 15, 20]  # Lista de grosores disponibles
paint_modes = ['normal', 'highlighter', 'oil']  # Modos de pintura
current_color = colors[0]
current_thickness = thicknesses[1]
current_paint_mode = paint_modes[0]

# Inicializar diccionario para el primer color, grosor y modo de pintura
if (current_color, current_thickness, current_paint_mode) not in points_dict:
    points_dict[(current_color, current_thickness, current_paint_mode)] = []

# Banderas para las interfaces
color_interface_active = False
thickness_interface_active = False
paint_mode_interface_active = False

# Función para interpolar los puntos y dibujar líneas continuas
def interpolate_points(p1, p2):
    points = []
    x1, y1 = p1
    x2, y2 = p2
    distance = int(((x2 - x1)**2 + (y2 - y1)**2)**0.5)
    if distance == 0:
        return [p1]
    for i in range(1, distance + 1):
        t = i / distance
        x = int(x1 + (x2 - x1) * t)
        y = int(y1 + (y2 - y1) * t)
        if (x, y) not in points:
            points.append((x, y))
    return points





def draw_oil_painting(img, points, thickness, color):
    overlay = img.copy()  # Capa de superposición para pintar
    mask = np.zeros_like(img, dtype=np.uint8)  # Máscara para las áreas pintadas

    for point in points:
        if point is None:
            continue
        
        # Variación del color para simular textura de óleo (bordes suaves y mezcla de colores)
        for offset in range(1, 3):  # Varias capas de bordes
            blur_color = tuple(min(255, max(0, c + np.random.randint(-20, 20))) for c in color)  # Variación de color
            cv2.circle(overlay, point, thickness // 2 + offset, blur_color, 1)  # Contornos difusos

        # Crear la máscara para las áreas pintadas
        cv2.circle(mask, point, thickness // 2, (255, 255, 255), -1)

    # Aplicar desenfoque solo en las áreas de la máscara
    blurred_overlay = cv2.GaussianBlur(overlay, (thickness * 2 + 1, thickness * 2 + 1), thickness / 2)
    img[mask[:, :, 0] > 0] = blurred_overlay[mask[:, :, 0] > 0]  # Actualizar solo las áreas pintadas

    # Superponer la textura de óleo solo en las áreas pintadas
    texture = np.random.randint(150, 255, img.shape[:2], dtype=np.uint8)  # Crear textura base
    texture = cv2.GaussianBlur(texture, (21, 21), 10)  # Suavizar textura
    texture_overlay = cv2.merge([texture, texture, texture])

    # Combinar la textura con las áreas pintadas
    alpha_texture = 0.2  # Ajustar la intensidad de la textura
    img[mask[:, :, 0] > 0] = cv2.addWeighted(
        texture_overlay[mask[:, :, 0] > 0],
        alpha_texture,
        img[mask[:, :, 0] > 0],
        1 - alpha_texture,
        0
    )

def draw_paint_mode(img, points, mode, thickness, color):
    if mode == 'normal':
        for point in points:
            if point is None:
                continue
            cv2.circle(img, point, thickness // 2, color, -1)

    elif mode == 'highlighter':
        overlay = img.copy()
        bright_color = tuple(min(255, int(c * 5.0)) for c in color)  # Hacer el color más brillante
        for point in points:
            if point is None:
                continue
            cv2.circle(overlay, point, thickness // 2, bright_color, -1)
        alpha = 0.6  # Transparencia para subrayador
        cv2.addWeighted(overlay, alpha, img, 1 - alpha, 0, img)

    elif mode == 'oil':
        draw_oil_painting(img, points, thickness, color)



# Bucle principal
while True:
    # Leer el frame de la cámara
    success, img = cap.read()
    if not success:
        break

    # Detectar manos
    hands, img = detector.findHands(img)

    # Dibujar las líneas entre puntos en la pantalla
    for (color, thickness, mode), points in points_dict.items():
        draw_paint_mode(img, points, mode, thickness, color)

    if hands:
        hand = hands[0]  # Solo la primera mano detectada
        lmList = hand["lmList"]  # Lista de posiciones (landmarks)

        # Coordenadas del dedo índice (punta)
        x_index, y_index = lmList[8][:2]  # Índice

        # Coordenadas del pulgar (punta)
        x_thumb, y_thumb = lmList[4][:2]  # Pulgar

        x = (x_index + x_thumb) // 2
        y = (y_index + y_thumb) // 2

        # Distancia entre el índice y el pulgar
        length, _, _ = detector.findDistance(lmList[8][:2], lmList[4][:2], img)

        # Detectar si la mano está cerrada (puño) para borrar
        fingers_down = all(lmList[f][1] > lmList[f - 2][1] for f in [8, 12, 16, 20])
        if fingers_down and not (color_interface_active or thickness_interface_active or paint_mode_interface_active):
            # Dibujar el círculo de borrado
            cv2.circle(img, (x, y), current_thickness, (0, 255, 255), -1)  # Borde amarillo
            cv2.circle(img, (x, y), current_thickness - 2, (0, 0, 0), -1)  # Interior negro
            for (color, thickness, mode), points in points_dict.items():
                points[:] = [point for point in points if point is None or ((x - point[0])**2 + (y - point[1])**2)**0.5 > 10]

        elif not color_interface_active and not thickness_interface_active and not paint_mode_interface_active and lmList[8][1] < lmList[6][1]:
            # Activar interfaz de color, grosor o modo de pintura según posición del dedo
            if x_index < img.shape[1] // 3:
                color_interface_active = True
            elif x_index < img.shape[1] * 2 // 3:
                thickness_interface_active = True
            else:
                paint_mode_interface_active = True

        elif not (color_interface_active or thickness_interface_active or paint_mode_interface_active):  # Solo permitir dibujar si ninguna interfaz está activa
            if length < 50:
                if (current_color, current_thickness, current_paint_mode) not in points_dict:
                    points_dict[(current_color, current_thickness, current_paint_mode)] = []
                if not drawing:
                    points_dict[(current_color, current_thickness, current_paint_mode)].append(None)
                drawing = True
                if len(points_dict[(current_color, current_thickness, current_paint_mode)]) > 0:
                    last_point = points_dict[(current_color, current_thickness, current_paint_mode)][-1]
                    if last_point is not None:
                        interpolated_points = interpolate_points(last_point, (x, y))
                        points_dict[(current_color, current_thickness, current_paint_mode)].extend(interpolated_points)
                    else:
                        points_dict[(current_color, current_thickness, current_paint_mode)].append((x, y))
            else:
                drawing = False

        # Interfaz de selección de color
        if color_interface_active:
            color_rectangles = []
            num_colors = len(colors)
            rect_width = 60
            spacing = 20
            start_x = (img.shape[1] - (num_colors * rect_width + (num_colors - 1) * spacing)) // 2
            start_y = img.shape[0] - 100

            for i, color in enumerate(colors):
                x1, y1 = start_x + i * (rect_width + spacing), start_y
                cv2.rectangle(img, (x1, y1), (x1 + rect_width, y1 + 50), color, -1)
                color_rectangles.append((color, (x1, y1, rect_width, 50)))

            if length < 50:
                for color, (x1, y1, w, h) in color_rectangles:
                    if x1 <= x <= x1 + w and y1 <= y <= y1 + h:
                        current_color = color
                        color_interface_active = False
                        break

        # Interfaz de selección de grosor
        if thickness_interface_active:
            thickness_circles = []
            num_thicknesses = len(thicknesses)
            spacing = 20  # Espacio entre los círculos
            start_x = (img.shape[1] - (num_thicknesses * (max(thicknesses) * 2 + spacing) - spacing)) // 2
            start_y = img.shape[0] - 200

            for i, thickness in enumerate(thicknesses):
                center_x = start_x + i * (thickness * 2 + spacing)  # Ajuste de posición según el grosor
                center_y = start_y
                # Dibujar un círculo con borde negro y relleno con el color actual
                cv2.circle(img, (center_x, center_y), thickness, (0, 0, 0), -1)  # Borde negro
                cv2.circle(img, (center_x, center_y), thickness - 1, current_color, -1)  # Relleno con el color actual
                thickness_circles.append((thickness, (center_x, center_y, thickness)))

            # Detectar si el clic está dentro de algún círculo
            if length < 50:
                for thickness, (center_x, center_y, radius) in thickness_circles:
                    if (x - center_x) ** 2 + (y - center_y) ** 2 <= radius ** 2:  # Verificar si el clic está dentro del círculo
                        current_thickness = thickness
                        thickness_interface_active = False
                        break

        # Interfaz de selección de modo de pintura
        if paint_mode_interface_active:
            mode_rectangles = []
            num_modes = len(paint_modes)
            rect_width = 60
            spacing = 20
            start_x = (img.shape[1] - (num_modes * rect_width + (num_modes - 1) * spacing)) // 2
            start_y = img.shape[0] - 300

            for i, mode in enumerate(paint_modes):
                x1, y1 = start_x + i * (rect_width + spacing), start_y
                cv2.rectangle(img, (x1, y1), (x1 + rect_width, y1 + 50), (200, 200, 200), -1)
                cv2.putText(img, f"{mode}", (x1 + 10, y1 + 35), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
                mode_rectangles.append((mode, (x1, y1, rect_width, 50)))

            if length < 50:
                for mode, (x1, y1, w, h) in mode_rectangles:
                    if x1 <= x <= x1 + w and y1 <= y <= y1 + h:
                        current_paint_mode = mode
                        paint_mode_interface_active = False
                        break

    # Mostrar la imagen resultante
    cv2.imshow("Dibujo", img)

    # Salir con la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
